In [1]:
%load_ext autoreload
%autoreload 2
import warnings; warnings.simplefilter('ignore')

import ipywidgets as widgets
from ipywidgets import interact, interact_manual, interactive
import seaborn as sns
import matplotlib.pyplot as plt
import os
import pandas as pd
from utils1 import *
data_path = os.getenv('HOME')+'/projects/beiwe-data/GxXEPM08ZK0GS1gIaLe9YhEn/1s3g19f7'

# Data Tables

In [2]:
feature_list = ['accel', 'callLog', 'tapsLog', 'usage', 'accessibilityLog', 'gps', 'light', 'powerState', 'textsLog']
user_map = {'Nikolas old ZTE':'16kbga47', 'Nikolas':'1s3g19f7', 'IMH1-Judy':'33kr56tx',
            'IMH2-Amirah':'1tfan3jn', 'Nikolas old Nokia':'8e3ukdwy', 'Praveen':'d35pt9m4',
            'Faye':'drdlfo5c', 'Robert':'gqmrnhvv', 'Xuancong':'hcy9th57'}


data_categories = feature_list
data_categories

['accel',
 'callLog',
 'tapsLog',
 'usage',
 'accessibilityLog',
 'gps',
 'light',
 'powerState',
 'textsLog']

In [3]:
data_path

'/Users/xuancong/projects/beiwe-data/GxXEPM08ZK0GS1gIaLe9YhEn/1s3g19f7'

In [4]:
@interact(cat=data_categories)
def show_data(cat='tapsLog'):
    global select_df
    df = create_df(data_path, cat)
    select_df = df.head()

interactive(children=(Dropdown(description='cat', index=2, options=('accel', 'callLog', 'tapsLog', 'usage', 'a…

In [5]:
data = select_df.widget.result
print('Start date: ', min(data['datetime']))
print('End date: ', max(data['datetime']))

AttributeError: 'DataFrame' object has no attribute 'widget'

# Data Visualization

# Call Log

In [ ]:
data = create_df(data_path, 'callLog')
def select_dates(start_date, end_date):
    start_date = localize_date(start_date)
    end_date = localize_date(end_date)
    mask = (data['datetime'] > start_date) & (data['datetime'] <= end_date)
    selection = data.loc[mask]
    selection.sort_values(by='datetime', inplace=True)
    selection.reset_index(inplace=True, drop=True)
    table = selection[['hashed phone number', 'call type', 'duration in seconds',
       'datetime', 'weekday_name', 'hour']]
    return table

date_picker = interact(select_dates,
                        start_date=widgets.DatePicker(value=pd.to_datetime('2019-08-05')),
                        end_date=widgets.DatePicker(value=pd.to_datetime('2019-09-09')), 
                        )
callLog_selection = date_picker.widget.result

### Phone call contacts
How many calls received form each hashed phone number

In [ ]:
callLog_selection['hashed phone number'].value_counts().reset_index(name='calls')

### Phone call types
Types of calls received form each hashed phone number.  
Note that the number appears more than once to match withe the respective call type

In [ ]:
callLog_selection.groupby(['hashed phone number', 'call type']).size().reset_index(name='count')

### Apps Usage

In [ ]:
data = create_df(data_path, 'tapsLog')
def select_dates(start_date, end_date):
    start_date = localize_date(start_date)
    end_date = localize_date(end_date)
    mask = (data['datetime'] > start_date) & (data['datetime'] <= end_date)
    
    selection = data.loc[mask]
    return selection, plot_apps_usage(selection, start_date, end_date)

date_picker = interact(select_dates,
                        start_date=widgets.DatePicker(value=pd.to_datetime('2019-08-05')),
                        end_date=widgets.DatePicker(value=pd.to_datetime('2019-09-09')), 
                        )

### Phone Usage based on taps data

In [ ]:
taps_selection = date_picker.widget.result[0]
plot_measurement_heatmap(taps_selection, 'tapsLog', norm=False)

In [ ]:
taps_selection
print('Start date: ', min(taps_selection['datetime']))
print('End date: ', max(taps_selection['datetime']))

### GPS trajectory

In [ ]:
gps_data = create_df(data_path, 'gps')

def select_dates(start_date, end_date):
    start_date = localize_date(start_date)
    end_date = localize_date(end_date)
    mask = (gps_data['datetime'] > start_date) & (gps_data['datetime'] <= end_date)
    selection = gps_data.loc[mask]
    selection.sort_values(by='datetime', inplace=True)
    selection.reset_index(inplace=True, drop=True)

    x,y = selection['longitude'], selection['latitude']
    
    
    return selection[['latitude', 'longitude', 'datetime']], gps_heatmap(x, y), 

date_picker = interact(select_dates,
                        start_date=widgets.DatePicker(value=pd.to_datetime('2019-08-20')),
                        end_date=widgets.DatePicker(value=pd.to_datetime('2019-09-09')), 
                        )


Faye's GPS data had some noise in the West of SG. Dates: 19 Aug to 20 Aug, check data around 12pm

# TODO
### GPS 
* Identify frequent vs non-frequent trips
* Transform coordinates with tester's consent

### Taps-Accesibility
* Find times the user presses delete / taps in a specific app

# Further data introspections